## Imports

### Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from epw import epw

### Test data

In [2]:
ashrae_measurements = pd.read_csv(
    "https://github.com/CenterForTheBuiltEnvironment/ashrae-db-II/raw/refs/heads/master/v2.1.0/db_measurements_v2.1.0.csv.gz",
    low_memory=False,
).reset_index(drop=True)

## Plots

In [3]:
from pythermalcomfort import theme

from pythermalcomfort.theme import (
    index_mapping_dictionary,
    tight_margins,
)

### Adaptive Model

In [4]:
from pythermalcomfort.utilities import operative_tmp
from pythermalcomfort.models import adaptive_ashrae

adaptive_subset = ashrae_measurements[["ta", "tr", "t_out", "vel"]].dropna()

adaptive_subset["top"] = operative_tmp(
    tdb=adaptive_subset["ta"],
    tr=adaptive_subset["tr"],
    v=adaptive_subset["vel"]
)

adaptive_results = adaptive_ashrae(
    tdb=adaptive_subset["ta"].to_numpy(),
    tr=adaptive_subset["tr"].to_numpy(),
    t_running_mean=adaptive_subset["t_out"].to_numpy(), # ! simplified for now
    v=adaptive_subset["vel"].to_numpy(),
    units="SI"
)

adaptive_subset["adaptive_acceptability_80%"] = adaptive_results["acceptability_80"]
adaptive_subset["adaptive_acceptability_90%"] = adaptive_results["acceptability_90"]

#### Classic chart

In [5]:
from pythermalcomfort.charts import adaptive_chart

df = adaptive_subset.sample(500, random_state=40)

adaptive_chart(df)

#### Alternative: With Summary

In [6]:
from pythermalcomfort.charts import adaptive_chart_with_summary

adaptive_chart_with_summary(df)

## PMV

In [7]:
pmv_subset = ashrae_measurements.loc[:, ["ta", "rh"]].dropna()

In [8]:
from pythermalcomfort.charts import pmv_chart

df = pmv_subset.sample(500, random_state=40)

pmv_inputs = {
    "met": 1.1,
    "clo": 0.5,
    "tr": 23,
    "v": 0.1,
}

pmv_chart(df, pmv_inputs)

## Temperature Heatmaps
### Example: Heat Index

In [9]:
weather_file = epw() # https://github.com/building-energy/epw
weather_file.read(
    "test_data/USA_AZ_Phoenix-Sky.Harbor.Intl.AP.722780_TMYx.2009-2023.epw"
)

In [10]:
from pythermalcomfort.models import heat_index

weather_data = weather_file.dataframe

weather_data["Hour"] = weather_data["Hour"] - 1
weather_data["Year"] = 2019 

weather_data["DateTime"] = pd.to_datetime(
    weather_data[["Year", "Month", "Day", "Hour"]]
)
weather_data["Month Name"] = weather_data["DateTime"].dt.strftime("%b")


weather_data["HI"] = weather_data.apply(
    lambda row: (
        heat_index(tdb=row["Dry Bulb Temperature"], rh=row["Relative Humidity"])["hi"]
        if row["Dry Bulb Temperature"] >= 27
        else None
    ),
    axis=1,
)

In [11]:
def categorize_heat_index(x, si_ip):

    if si_ip.lower() == "ip":
        x = (x - 32) * 5 / 9

    if 27 <= x < 32:
        return "Caution"
    elif 32 <= x < 41:
        return "Extreme<br>Caution"
    elif 41 <= x < 54:
        return "Danger"
    elif x >= 54:
        return "Extreme<br>Danger"
    else:
        return "No Risk"


weather_data["categorical"] = weather_data["HI"].apply(
    categorize_heat_index, si_ip="si"
)

In [12]:
from pythermalcomfort.charts import heatmap

heatmap(df=weather_data, var="HI", global_local="global", si_ip="si")

In [13]:
from pythermalcomfort.charts import heatmap_with_summary

heatmap_with_summary(weather_data, "HI", global_local="global", si_ip="si")